<a href="https://colab.research.google.com/github/joshetuk/data-science-and-ai-/blob/main/satelliteremotesensing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Exercise 1: Client vs Server side operations on the GEE
#When working with the GEE we must distinguish between client side and server side operations. The difference is that a client side command
#is run locally on your system. Conversely, a server side command will create a chain of instructions on what to do when run by the server
#(Google's computers). This chain is only sent to the server when a result is requested. So the processing doesn't happen unless the output is
#requested
# Import the google earth engine API:
import ee# Trigger the authentication flow:


In [ ]:
# Trigger the authentication flow:
ee.Authenticate()

In [ ]:
# Initialize the library:
ee.Initialize(project='ee-joshuafredrick165') # where xxxx is the project-ID you wrote down when
# you created your Google Earth Engine Account

In [ ]:
# Import the geemap library:
import geemap

In [ ]:
# let's ask the GEE to construct a number:
b = ee.Number(2)

In [ ]:
# evaluates to a GEE object (ee_number) not an integer:
print(type(b))

<class 'ee.ee_number.Number'>


In [ ]:
# let's add something to it:
sum = b.add(1)
# .add(), .subtract() .divide() .multiply() are some of the arithmetic operators of the GEE

In [ ]:
print(sum)
# prints the details of the command we asked the GEE to perform not the result of the arithmetic operation
b == 2
# results to FALSE, remember b is an object defined on the server not a number
b.eq(2)
# this is the server side equivalent to "==" and evaluates as 1 (TRUE)


ee.Number({
  "functionInvocationValue": {
    "functionName": "Number.add",
    "arguments": {
      "left": {
        "constantValue": 2
      },
      "right": {
        "constantValue": 1
      }
    }
  }
})


In [ ]:
print(a + b.getInfo())
# Use .getInfo() to retrieve the server-side value of 'b' as a client-side integer before addition.
# This allows you to perform the addition operation with the integer 'a'.

4


In [ ]:
#exercise 2

In [ ]:
ROI = ee.Geometry.Polygon(
[[67.42, 26.54],
[68.65, 26.54],
[68.65, 27.73],
[67.42, 27.73]]
) #First define a Region of Interest (ROI) and send this to the GEE as a Geometry (polygon) dening the coordinates of its four corners

In [ ]:
#Then dene two periods of observation; one before the foods started and one during the event

pre_flood_start_date = '2021-08-01'
pre_flood_end_date = '2021-09-30'
flood_start_date = '2022-08-01'
flood_end_date = '2022-09-30'

In [ ]:
#Lets make an interactive map, add the region of interest as a layer and center the map on it
m = geemap.Map() # uses the geemap library to make an empty map
m.addLayer(ROI, # adds a layer to the map displaying the ROI
{'color':'green'}, # this is a dictionary setting the visualisation parameteres
'Region of Interest') # gives a name to the layer
m.centerObject(ROI,9) # centers the map to the ROI; 9 is the zoom level
m # displays the map
# SPEND A COUPE OF MINUTES USING THE INTERACTIVE TOOLS ON THE MAP WINDOW BELOW
# Try different zoom levels by changing the number on the m.centerObject command

Map(center=[27.135283876202667, 68.03500000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
#To access the entire ImageCollection for sentinel 1 data use:
S1Collection = ee.ImageCollection('COPERNICUS/S1_GRD')

In [ ]:
#The above command will give you all images ever collected by that satellite mission (hundrends of TB!). We use lters to select the
#appropriate images we need for our application. The following line will select only images that are collected with the Interferometric Wide
#Swath mode
S1Filtered = S1Collection.filter(ee.Filter.eq('instrumentMode', 'IW'))

In [ ]:
#Lets apply some more lters. We want images that have been acquired with VV (vertical transmit / vertical receive) and VH (vertical transmit
#/ horizontal receive) polarization, we want them to be at an ascending satellite orbit (when working with SAR data we don't mix ascending and
#descending orbit data)
S1Filtered = (
S1Filtered
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
.filterBounds(ROI)
)
# let's find out how many satellite images we have got
print('Number of images: ',S1Filtered.size().getInfo())

Number of images:  987


In [ ]:
#Now lets lter further to get one collection of images from the 'pre ood' period we dened and one for the 'during ood' period.
#Here we also use .select() to select only the VV and VH polarisation band of the images
S1Filtered2021=(
S1Filtered
.filterDate(pre_flood_start_date, pre_flood_end_date)
.select(['VV','VH'])
)
S1Filtered2022=(
S1Filtered
.filterDate(flood_start_date, flood_end_date)
.select(['VV','VH'])
)
# let's find out how many satellite images we have got on each image collection
print('Number of pre flood images: ',S1Filtered2021.size().getInfo())
print('Number of post flood images: ',S1Filtered2022.size().getInfo())

Number of pre flood images:  20
Number of post flood images:  20


In [ ]:
#In the next step we will reduce the image collection to a single image. We can do this by deriving a min, mean, median or max value for each
#pixel of the image
beforeImage = S1Filtered2021.mean()
duringImage = S1Filtered2022.mean()
# by using ".mean()" we convert a collection of images into a single image. The
# value of each pixel in the new image is equal to the mean value calculated
# from the image collection on a pixel by pixel basis
print(type(S1Filtered2021)) # this evaluates to an ee.imagecollection
print(type(beforeImage)) # this evaluates to an ee.image

<class 'ee.imagecollection.ImageCollection'>
<class 'ee.image.Image'>


In [ ]:
#Lets display the mean images on a new map
m = geemap.Map()
m.addLayer(beforeImage,{},'before mean image')
m.addLayer(duringImage,{},'during mean image')
m.addLayer(ROI,{'color': 'green'}, 'ROI')
m.centerObject(ROI,9)
m
# the empty curly brackets above {} are where we usually define
# image visualisation parameters (e.g. colour palette) for now we have left
# this emply but we will look at it later

Map(center=[27.135283876202667, 68.03500000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
#As you see on the map above the images generated expand beyond our Region of Interest. Lets clip the images to the desired extent
beforeImage = beforeImage.clip(ROI)
duringImage = duringImage.clip(ROI)
m = geemap.Map()
m.addLayer(beforeImage,{},'before mean image')
m.addLayer(duringImage,{},'during mean image')
m.centerObject(ROI,9)
m

Map(center=[27.135283876202667, 68.03500000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
#To enhance visualisation we will create a new band using the ratio of the VV and VH bands and specify how the images should be displayed
#by dening visualisation parameters "visParams" as a dictionary.
beforeImage = beforeImage.addBands(
beforeImage.select('VV').divide(beforeImage.select('VH')).rename('VV/VH'))
duringImage = duringImage.addBands(
duringImage.select('VV').divide(duringImage.select('VH')).rename('VV/VH'))
visParams = {
'min':[-25, -25, 0],
'max':[0,0,2]
}
m = geemap.Map()
m.centerObject(ROI,9)
m.addLayer(beforeImage,visParams,'before mean image')
m.addLayer(duringImage,visParams,'during mean image')
m
# TRY TO CHANGE LAYER VISIBILILTY (SPANNER ICON ON MAP WINDOW TOP RIGHT CORNER)
# WHAT DO YOU OBSERVE BETWEEN THE TWO LAYERS?

Map(center=[27.135283876202667, 68.03500000000001], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
#nally lets make a split panel map with both images to enhance comparison
left_layer = geemap.ee_tile_layer(beforeImage, visParams, 'before the flood')
right_layer = geemap.ee_tile_layer(duringImage, visParams, 'during the flood')
m.split_map(
left_layer, right_layer, left_label='Before', right_label='During'
)
m

Map(bottom=27862.0, center=[27.479034752500656, 66.76116943359376], controls=(ZoomControl(options=['position',…

In [ ]:
#Exercise 3: maping water quality changes on the Humber estuary
#Here we will use images from Sentinel-2 to look at variations of water quality of the Humber estuary. Specically we will focus on the amount#
#of turbidity (how muddy the water is) and the amount of chlorophyll.
#Exercise 3: maping water quality changes on the Humber estuary
#First lets dene our Area of Interest using a Polygon
AOI = ee.Geometry.Polygon([[-0.79, 53.75],
[-0.79, 53.55],
[0.16, 53.55],
[0.16, 53.75]])
m = geemap.Map()
m.addLayer(AOI,{},'Humber area')
m.centerObject(AOI)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
#The AOI dened above contains both water and land around the Humber. In this exercise we are only interested in water. For this reason we
#will import a Gobal Surface Water dataset that has been developed by the Joint Research Centre of the European Commission to further
#constrain our AOI
# Imports the global surface water dataset
gsw = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
Humber = (gsw
.clip(AOI) # Clip to the Area of Interest
.select('seasonality').gte(12) # Keep only pixels that have water all
# monhts of the year
.selfMask() # Mask out empty pixels
)
m = geemap.Map()
m.addLayer(Humber,{},'Humber estuary')
m.centerObject(AOI)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
#The above code produces a raster with pixels showing parts of the estuary that are identied as water. We would prefer to have a vector
#(polygon). Rasters can be converted to vectors in certain cases, lets do that
Humber = Humber.reduceToVectors()
m = geemap.Map()
m.addLayer(Humber,{},'Humber estuary')
m.centerObject(AOI)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
#The Sentinel-2 mission of satellites carries on board a multispectral passive sensor (camera) capturing information in 12 bands. Light
#emmitted from the Sun is passing through the atmosphere, reected by the Earth surface back through the atmosphere and is caputred by
#the sensor onboard of the satellite. The presence of the atmosphere creates errors in the recording that require corrections (atmospheric
#correction)
S2Collection= ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

In [ ]:
#Next we will lter the collection to get images for two distinct dates (4 April 2021 and 19 April 2021) we will also lter the collection to cover
#only the Humber esuary
S2CollectionA = (
S2Collection
.filterDate('2021-04-04', '2021-04-05')
.filterBounds(Humber)
)
S2CollectionB = (
S2Collection
.filterDate('2021-04-19', '2021-04-20')
.filterBounds(Humber)
)
print(S2CollectionA.size().getInfo())
print(S2CollectionB.size().getInfo())

3
3


In [ ]:
#The above code shows that there are three images in each collection. We can display the footprint of each image in an image collection by
#using ".geomerty()". By doing that we can see with the code below that we are dealing with one main image and two small pieces for
#neighbouring images.
m = geemap.Map()
m.centerObject(Humber)
m.addLayer(S2CollectionA.geometry(), {"color":'Green'}, 'S2 Collection A')
m.addLayer(S2CollectionB.geometry(), {"color":'Red'}, 'S2 Collection B')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
#Let's chose the rst image from each collection, as this happens to be the largest tile, and clip them to the area of interest we dened earlier
#(Humber). We can then assign visualisation parameters using the bands 4,3,2 (red, green, blue) and display the two images in a split panel
#map
S2ImageA = S2CollectionA.first().clip(Humber)
S2ImageB = S2CollectionB.first().clip(Humber)
visualization = {'bands':['B4','B3','B2'],
'min':146,
'max':2183,
'gamma':1.4}
m = geemap.Map()
m.centerObject(Humber)
left_layer = geemap.ee_tile_layer(S2ImageA,
visualization,
'4 April 2021' )
right_layer = geemap.ee_tile_layer(S2ImageB,
visualization,
'19 April 2021' )
m.split_map(
left_layer, right_layer,
left_label='4 April 2021', right_label='19 April 2021'
)
m
# Do you see a difference in the colour of the two images?

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [ ]:
#Now lets calculate the Normalised Difference Turbidity Index (NDTI) for the two images. The NDTI uses the reectance on the green (band 3)
#and red (band 4) of the Electromagnetic Specturm and is used as an indicator of turbidity in water bodies. Values range between -1 and 1 with
#higher values indicating higher turbidity. We will make a split panel map to see how turbidity content changed between the two dates
# First calculate the normalised difference of the two bands and
# add it into the image as a new band called NDTI
# for the first image:
ndtiA = S2ImageA.normalizedDifference(['B4','B3']).rename('NDTI')
# for the second image:
ndtiB = S2ImageB.normalizedDifference(['B4','B3']).rename('NDTI')
# speficy the visualisation parameters for the NDVI
visParams = {'min':-0.15, 'max':0.15, 'palette':['lightblue','white','brown']}
#make a split panel map
m = geemap.Map()
m.centerObject(Humber)
left_layer = geemap.ee_tile_layer(ndtiA, visParams,'NDTI A')
right_layer = geemap.ee_tile_layer(ndtiB,visParams,'NDTI B')
m.split_map(
left_layer, right_layer,
left_label='4 April 2021', right_label='19 April 2021'
)
# add a colorbar
m.add_colorbar(
vis_params=visParams,
label='NDTI',
position='bottomright'
)
m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [ ]:
#Now repeat the same for the Normalised Difference Chlorophyll Index (NDCI). The NCTI uses the reectance on the rededge (band 5) and red
#(band 4) parts of the Electromagnetic Specturm and is used as an indicator of Chlorophyll in water bodies. Values range between -1 and 1
#with higher values indicating higher Chlorphyll
# for the first image:
ndtiA = S2ImageA.normalizedDifference(['B5','B4']).rename('NDCI')
# for the second image:
ndtiB = S2ImageB.normalizedDifference(['B5','B4']).rename('NDCI')
# speficy the visualisation parameters for the NDVI
visParams = {'min':-0.15, 'max':0.15, 'palette':['lightblue','white','brown']}
#make a split panel map
m = geemap.Map()
m.centerObject(Humber)
left_layer = geemap.ee_tile_layer(ndtiA, visParams,'NDCI A')
right_layer = geemap.ee_tile_layer(ndtiB,visParams,'NDCI B')
m.split_map(
left_layer, right_layer,
left_label='4 April 2021', right_label='19 April 2021'
)
# add a colorbar
m.add_colorbar(
vis_params=visParams,
label='NDCI',
position='bottomright'
)
m


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…